<a href="https://colab.research.google.com/github/Nobobi-Hasan/FND-Llama/blob/main/FND_Llama_01_RAG_1_00_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1: Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from huggingface_hub import login
login()

# 2: Install Dependencies

In [3]:
!pip install -q langchain-core langchain-community langchain-chroma \
langchain-text-splitters unstructured transformers bitsandbytes accelerate \
sentence-transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Cell 3: Create Dummy Knowledge Base

In [4]:
# This simulates "Wikipedia and fact-checking databases"
!mkdir -p data
!echo "The Earth is an oblate spheroid. It is the third planet from the Sun." > data/knowledge_base.txt
!echo "A study by the Flat Earth Society claims the Earth is a flat disc." >> data/knowledge_base.txt
!echo "Scientific consensus, based on satellite imagery and physics, confirms Earth is round." >> data/knowledge_base.txt

# Cell 4: Imports

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, logging
from peft import PeftModel
from langchain.llms import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

Cell 5: Configuration

In [6]:
# The path to saved ADAPTER checkpoint
ADAPTER_PATH = "/content/drive/MyDrive/ML-Models/FND_Llama_01_Fine_Tuning_1.00/checkpoint-epoch-1"

# The name of the base model
BASE_MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

# Path to the knowledge base
KNOWLEDGE_BASE_PATH = "data/knowledge_base.txt"

# Embedding model to use
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

# Statement to Test
test_statement = "The Earth is flat."

Cell 6: Helper Function to Load Model

In [11]:
# function for loading the base model and attach the adapter

def load_model_for_inference(base_model_name, adapter_path):
    # 1. Load the BASE model in 4-bit
    print(f"Loading base model: {base_model_name}...")

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=quant_config,
        device_map="auto",
        trust_remote_code=True
    )
    print("Base model loaded.")

    # Load the PEFT adapter from Google Drive
    print(f"Loading adapter from: {adapter_path}")
    model = PeftModel.from_pretrained(model, adapter_path)
    print("Adapter attached.")

    return model, tokenizer

Cell 7: LOAD CLASSIFIER

In [14]:
print("Loading Classifier Model...")

# Suppress warnings
logging.set_verbosity(logging.CRITICAL)

# Call the helper function
classifier_model, classifier_tokenizer = load_model_for_inference(BASE_MODEL_NAME, ADAPTER_PATH)

classifier_pipe = pipeline(
    "text-generation",
    model=classifier_model,
    tokenizer=classifier_tokenizer,
    max_new_tokens=10 # For classification
)
print("Classifier loaded.")

Loading Classifier Model...
Loading base model: meta-llama/Llama-3.2-3B-Instruct...


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Base model loaded.
Loading adapter from: /content/drive/MyDrive/ML-Models/FND_Llama_01_Fine_Tuning_1.00/checkpoint-epoch-1
Adapter attached.
Classifier loaded.


Cell 8: LOAD RAG VERIFIER

In [15]:
print("Loading RAG Verifier...")

# 8a. Load embeddings
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

# 8b. Load and chunk documents
loader = TextLoader(file_path=KNOWLEDGE_BASE_PATH)
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs_split = splitter.split_documents(docs)

# 8c. Create vector store and retriever
vectorstore = Chroma.from_documents(documents=docs_split, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# 8d. Wrap classifier model in LangChain for RAG
rag_llm = HuggingFacePipeline(pipeline=classifier_pipe)

# 8e. Define RAG prompt
rag_prompt = ChatPromptTemplate.from_template(
    """
    You are a verification assistant. Use the context to verify the statement.
    Respond with only "Real" or "Fake".

    Context:
    {context}

    Statement:
    {question}

    Answer:
    """
)

# 8f. Create RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | rag_llm
    | StrOutputParser()
)
print("RAG Verifier loaded.")

Loading RAG Verifier...


/tmp/ipython-input-479838296.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RAG Verifier loaded.


/tmp/ipython-input-479838296.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  rag_llm = HuggingFacePipeline(pipeline=classifier_pipe)
